# Simulation MFD

In this tutorial we introduce a full demonstration of a simulation with `mnms` from the creation of the `MultiModalGraph` to a simulation run with the `MFDFlow` motor.
A quick demonstration of results analysis is also made at the end of the notebook.

## Simulation

### MultiModalGraph

In [ ]:
from mnms.graph import MultiModalGraph
from mnms.graph.edition import walk_connect
from mnms.graph.generation import create_grid_graph
from mnms.mobility_service import BaseMobilityService

mmgraph = create_grid_graph(10, 5, 1000)
mmgraph.add_zone('ZONE', [l.id for l in mmgraph.flow_graph.links.values()])

car = BaseMobilityService('car', 10)
bus = BaseMobilityService('bus', 10) 

for n in mmgraph.flow_graph.nodes.keys():
    car.add_node('CAR_'+n, n)
    bus.add_node('BUS_'+n, n)

for l in mmgraph.flow_graph.links.values():
    uid = l.upstream_node
    did = l.downstream_node
    car.add_link('CAR_'+uid+'_'+did, 'CAR_'+uid, 'CAR_'+did, {'length': 1000, 'time':1000/car.default_speed}, [l.id])
    bus.add_link('BUS_' + uid + '_' + did, 'BUS_' + uid, 'BUS_' + did, {'length': 1000, 'time':1000/bus.default_speed}, [l.id])

mmgraph.add_mobility_service(car)
mmgraph.add_mobility_service(bus)
mmgraph.mobility_graph.check()

walk_connect(mmgraph, 1)

In [ ]:
from mnms.tools.render import draw_multimodal_graph
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 9))
draw_multimodal_graph(ax, mmgraph, nodesize=5, linkwidth=2, dy=2000)

### Demand

Creation of simple `BaseDemandManager` and we attach a `CSVUserObserver` to get more info in CSV file about the travel of all the `Users`

In [ ]:
from mnms.demand.generation import create_random_demand
from mnms.tools.observer import CSVUserObserver

demand = create_random_demand(mmgraph, "07:00:00", "10:00:00", cost_path='length', min_cost=5000, seed=42)
demand.add_user_observer(CSVUserObserver("results/user_observer.csv"))

### Travel decision model

In [ ]:
from mnms.travel_decision import LogitDecisionModel
travel_decision = LogitDecisionModel(mmgraph, outfile="results/path.csv")

### Flow motor

In [ ]:
from mnms.flow.MFD import Reservoir, MFDFlow

def res_fct(dict_accumulations):
    V_car = 11.5 * (1 - (dict_accumulations['car'] + dict_accumulations['bus']) / 500)
    V_car = max(V_car, 0.001)
    V_bus = V_car/2
    dict_speeds = {'car': V_car, 'bus': V_bus}
    return dict_speeds


reservoir = Reservoir.fromZone(mmgraph, 'ZONE', res_fct)

flow_motor = MFDFlow(outfile="results/flow.csv")
flow_motor.add_reservoir(reservoir)

### Supervisor

In [ ]:
from mnms.simulation import Supervisor
from mnms.tools.time import Time, Dt

supervisor = Supervisor(graph=mmgraph,
                        flow_motor=flow_motor,
                        demand=demand,
                        decision_model=travel_decision,
                        outfile="results/travel_time_link.csv")

supervisor.run(Time('06:55:00'), 
               Time('10:20:00'), 
               Dt(minutes=1), 
               10)

## Result analysis

### Flow analysis

In [ ]:
import pandas as pd

flow_result = pd.read_csv('results/flow.csv', sep=';')
flow_result

In [ ]:
flow_result_bus = flow_result[flow_result['MODE']=='bus']
flow_result_car = flow_result[flow_result['MODE']=='car']

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(16, 9))

ax.plot(flow_result_bus['TIME'].to_list(), flow_result_bus['ACCUMULATION'].to_list(), 'r+-', label='BUS')
ax.plot(flow_result_car['TIME'].to_list(), flow_result_car['ACCUMULATION'].to_list(), 'b+-', label='CAR')
ax.xaxis.set_major_locator(plt.MaxNLocator(20))
plt.grid()
plt.legend()
fig.autofmt_xdate()

### User path analysis

In [ ]:
user_results = pd.read_csv('results/user_observer.csv', sep=';')
user_results

In [ ]:
hist = user_results.hist(column="COST_PATH", 
                  figsize=(16, 9), 
                  bins=100)
_ = plt.xlabel('cost path')